In [1]:
from parse import get_data
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
# from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from parse import sample_data
import re

Using TensorFlow backend.


In [2]:
train, labels = get_data("data/corpus.data")
nb_of_entries = len(train)
print(nb_of_entries)
df = pd.DataFrame({'Text' : train, 'Tag': labels})



max_features = 100000
tk = Tokenizer(num_words=max_features, split=' ')
tk.fit_on_texts(df['Text'].values)
X = tk.texts_to_sequences(df['Text'].values)
# print(X)
X = pad_sequences(X)

def result_predict(resultarray) :
    if resultarray[0] == resultarray.max() :
        return "irr"
    if resultarray[1] == resultarray.max() :
        return "neg"
    if resultarray[2] == resultarray.max() :
        return "neu"
    if resultarray[3] == resultarray.max() :
        return "pos"

4998


In [3]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())

Y = pd.get_dummies(df['Tag']).values
percentage_train = 0.9
borne = int(percentage_train*nb_of_entries)
X_train = X[:borne]
Y_train = Y[:borne]
X_test = X[borne:]
Y_test = Y[borne:]

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

batch_size = 200
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

(4498, 48) (4498, 4)
(500, 48) (500, 4)
Epoch 1/15
4498/4498 [==============================] - 11s 3ms/step - loss: 1.2154 - acc: 0.4469
Epoch 2/15
4498/4498 [==============================] - 11s 3ms/step - loss: 1.0720 - acc: 0.5169
Epoch 3/15
4498/4498 [==============================] - 12s 3ms/step - loss: 0.9386 - acc: 0.6423
Epoch 4/15
4498/4498 [==============================] - 12s 3ms/step - loss: 0.7749 - acc: 0.6956
Epoch 5/15
4498/4498 [==============================] - 14s 3ms/step - loss: 0.6067 - acc: 0.7692
Epoch 6/15
4498/4498 [==============================] - 11s 3ms/step - loss: 0.4665 - acc: 0.8330
Epoch 7/15
4498/4498 [==============================] - 11s 3ms/step - loss: 0.3604 - acc: 0.8784
Epoch 8/15
4498/4498 [==============================] - 12s 3ms/step - loss: 0.2761 - acc: 0.9062
Epoch 9/15
4498/4498 [==============================] - 13s 3ms/step - loss: 0.2216 - acc: 0.9242
Epoch 10/15
4498/4498 [==============================] - 12s 3ms/step - loss: 

In [11]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print(score, acc)

1.6082910482977093 0.5619335291608943


In [7]:

list_to_test = sample_data("tw_db_prepared.data", 10)

for text in list_to_test :
    print(text)
    twt = tk.texts_to_sequences(text)
    twt = pad_sequences(twt, maxlen=62, dtype='int32', value=0)
    sentiment=model.predict(twt, batch_size=1, verbose=2)[0]
    print(result_predict(sentiment))
    

revoir les taux votes par csp niveau revenu lors des législatives tant que les classesmoyennes sup sont pas lessivées comme les gilets jaunes par macron président des ultras riches lrem sera devant attendre ans désindexation retraite 
pos
vous êtes une grande naïveté monsieur ensorcelé par gourou soudaine empathie pour ceux qui sont rien vous surprend pas cette période approchant les élections européennes macron essaie soigner son image arrogant hors sol 
pos
macron fera jamais rien qui puisse choquer les musulmans les islamistes car est une part importante son électorat trahison 
pos
dictature régime politique dans lequel tous les pouvoirs sont entre les mains une seule personne oui effectivement macron est pas seul ceux qui ont intérêt que reste comme aide bcp vrai décide pas chose 
pos
affaire macron benalla réponse nos détracteurs via 
pos
est macron qui est complice pire
pos
bref jsuis vener cause vos histoires plus macron fait des bottle flip pendant que samedi peux plus sortir c

In [13]:
twt = ['macron est le meilleur président, les autres candidats n\'ont aucun chance']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tk.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=39, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(result_predict(sentiment))
print(sentiment)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    1    3   31    2   96 4727  254  877]]
neg
[0.11338186 0.544712   0.10707022 0.23483588]
